In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/boaz/

/content/drive/MyDrive/boaz


In [21]:
import pandas as pd
df = pd.read_csv("credit_card_data_city.csv")
df

,pop_cd,year,month,avg_score,avg_rat,city,ages,population,num_opencard,num_usecard,...,monthly_installments_loan,monthly_insurance_loan,monthly_sbk_loan,loan_commitment,inst_rep_loanb,ls_rep_loanb,credit_loan,mortgage_loan,credit_card_payment,credit_card_installments_payment
0,L011,2016,1,812,2,서울,10대,55000,1.44,1.13,...,0,0,0,1120000,8210000,78020000,92170000,87260000,0,0
1,L011,2016,2,773,4,서울,10대,380000,1.71,0.96,...,510000,220000,1020000,680000,4660000,23300000,8260000,42410000,4000,1000
2,L011,2016,3,771,4,서울,10대,387000,1.75,0.95,...,490000,210000,1140000,640000,4370000,22930000,8300000,41710000,3000,1000
3,L011,2016,4,770,4,서울,10대,412000,1.76,0.92,...,510000,240000,1340000,510000,4770000,22160000,9460000,41040000,4000,1000
4,L011,2016,5,771,4,서울,10대,414000,1.75,0.94,...,490000,340000,1490000,640000,4740000,21960000,9660000,40850000,3000,1000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3451,L169,2017,8,810,3,제주,90대,2000,0.38,0.07,...,0,0,0,3000000,6990000,18960000,17060000,35400000,15000,7000
3452,L169,2017,9,809,3,제주,90대,2000,0.41,0.06,...,0,0,0,2830000,7640000,18970000,17940000,34670000,18000,1000
3453,L169,2017,10,809,3,제주,90대,2000,0.42,0.06,...,0,0,0,2940000,7910000,16000000,17940000,30240000,13000,8000
3454,L169,2017,11,810,3,제주,90대,2000,0.43,0.05,...,0,0,0,2780000,7420000,31050000,17560000,51510000,15000,7000


### __For Sankey__

In [22]:
#총 대출 금액(monthly_total_loan) 칼럼 새로 생성
df['monthly_total_loan'] = df[[#'monthly_bk_loan', #은행대출=신용대출+담보대출
                               'credit_loan', #신용대출
                               'mortgage_loan', #담보대출
                               'monthly_cd_loan', #카드대출
                               'monthly_installments_loan', #할부대출
                               'monthly_insurance_loan', #보험대출
                               'monthly_sbk_loan' #저축은행대출
                               ]].apply(lambda x: x.sum(), axis=1)

In [23]:
#sankey 차트를 만들 수 있는 form으로 변형: wide -> long by melt
tmp = df[[#'monthly_bk_loan', 
          'credit_loan', 
          'mortgage_loan',
          'monthly_cd_loan', 
          'monthly_installments_loan',
          'monthly_insurance_loan', 
          'monthly_sbk_loan', 
          'monthly_total_loan']]
tmp = tmp.melt(id_vars='monthly_total_loan')

In [24]:
#최종 데이터프레임 정리
tmp = tmp.rename(columns={'monthly_total_loan': '대출총금액',
                          'variable': '대출중분류',
                          'value': '대출금액'})
tmp['대출대분류'] = '총대출금액'
tmp

,대출총금액,대출중분류,대출금액,대출대분류
0,179430000,credit_loan,92170000,총대출금액
1,52610000,credit_loan,8260000,총대출금액
2,52020000,credit_loan,8300000,총대출금액
3,52780000,credit_loan,9460000,총대출금액
4,53040000,credit_loan,9660000,총대출금액
...,...,...,...,...
20731,52460000,monthly_sbk_loan,0,총대출금액
20732,52610000,monthly_sbk_loan,0,총대출금액
20733,48180000,monthly_sbk_loan,0,총대출금액
20734,69070000,monthly_sbk_loan,0,총대출금액


In [25]:
#저장
tmp.to_csv('credit_card_for_sankey2.csv', encoding='utf-8-sig', index=False)

### __For Donut__

In [ ]:
#도넛차트에 이용할 칼럼 생성
df['check_card_payment'] = df.apply(lambda x: x['monthly_card_spend']-x['credit_card_payment'], axis=1)
df['credit_noinstallments_payment'] = df.apply(lambda x: x['credit_card_payment']-x['credit_card_installments_payment'], axis=1)

In [ ]:
#도넛차트에 이용할 칼럼 추출 및 형태 변형
tmp = df[['ages', 'check_card_payment', 'credit_card_payment', 'monthly_card_spend',
          'credit_noinstallments_payment', 'credit_card_installments_payment']]
tmp = tmp.melt(id_vars='ages')

In [ ]:
#저장
tmp.to_csv('credit_card_for_donut.csv', encoding='utf-8-sig', index=False)

### __For Hexmap__

In [ ]:
xy = pd.read_csv('add-list (map_hex_Korea)_add-list.csv', encoding='utf-8-sig')
xy

,ADD_sgguCdNm_full,ADD_sgguCdNm_GNR,ADD_sidoCdNm_full,CHK_GNR_GU,ID,ID_NEW,id_No,sgguCd,sgguCdNm,sidoCd,sidoCdNm,x,y,y_rev
0,양구군,양구군,강원도,N,양구,강원 양구,2,320004,양구군,320000,강원,9,0,27
1,고성군,고성군,강원도,N,고성(강원),강원 고성,3,320001,고성군,320000,강원,10,0,27
2,철원군,철원군,강원도,N,철원,강원 철원,0,320010,철원군,320000,강원,7,0,27
3,화천군,화천군,강원도,N,화천,강원 화천,1,320014,화천군,320000,강원,8,0,27
4,양주시,양주시,경기도,N,양주,경기 양주,4,312700,양주시,310000,경기,3,1,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,장흥군,장흥군,전라남도,N,장흥,전남 장흥,247,360018,장흥군,360000,전남,2,24,3
248,진도군,진도군,전라남도,N,진도,전남 진도,245,360019,진도군,360000,전남,0,24,3
249,제주시,제주시,제주특별자치도,N,제주,제주 제주,250,390200,제주시,390000,제주,5,25,2
250,완도군,완도군,전라남도,N,완도,전남 완도,249,360016,완도군,360000,전남,2,25,2


In [ ]:
tmp = pd.merge(df, xy, how='outer', left_on='city', right_on='sidoCdNm')
tmp

,pop_cd,year,month,avg_score,avg_rat,city,ages,population,num_opencard,num_usecard,...,ID,ID_NEW,id_No,sgguCd,sgguCdNm,sidoCd,sidoCdNm,x,y,y_rev
0,L011,2016.0,1.0,812.0,2.0,서울,10대,55000.0,1.44,1.13,...,서울 도봉,서울 도봉,14,110006,도봉구,110000,서울,5,2,25
1,L011,2016.0,1.0,812.0,2.0,서울,10대,55000.0,1.44,1.13,...,서울 노원,서울 노원,15,110022,노원구,110000,서울,6,2,25
2,L011,2016.0,1.0,812.0,2.0,서울,10대,55000.0,1.44,1.13,...,서울 성북,서울 성북,24,110012,성북구,110000,서울,6,3,24
3,L011,2016.0,1.0,812.0,2.0,서울,10대,55000.0,1.44,1.13,...,서울 강북,서울 강북,23,110024,강북구,110000,서울,5,3,24
4,L011,2016.0,1.0,812.0,2.0,서울,10대,55000.0,1.44,1.13,...,서울 종로,서울 종로,34,110016,종로구,110000,서울,6,4,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54212,L169,2017.0,11.0,810.0,3.0,제주,90대,2000.0,0.43,0.05,...,제주,제주 제주,250,390200,제주시,390000,제주,5,25,2
54213,L169,2017.0,11.0,810.0,3.0,제주,90대,2000.0,0.43,0.05,...,서귀포,제주 서귀포,251,390100,서귀포시,390000,제주,5,26,1
54214,L169,2017.0,12.0,811.0,3.0,제주,90대,2000.0,0.43,0.06,...,제주,제주 제주,250,390200,제주시,390000,제주,5,25,2
54215,L169,2017.0,12.0,811.0,3.0,제주,90대,2000.0,0.43,0.06,...,서귀포,제주 서귀포,251,390100,서귀포시,390000,제주,5,26,1


In [ ]:
tmp.to_csv('merge_for_hexmap.csv', encoding='utf-8-sig', index=False)